In [42]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
#retrieve all the postal code from website wikipedia

list_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(list_url).text

In [3]:
soup = BeautifulSoup(source, 'xml')
table = soup.find('table')

In [7]:
#building the dataframe into columns

column_name = ['Postal code', 'Borough', 'Neighborhood']
df= pd.DataFrame(columns= column_name)

In [8]:
#retrieve all the borough, neighborhood and postal code

for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


As specified in the assignment, some boroughs are not assigned. Those have to be removed so we will only be taking care of the ones assigned.

In [9]:
#clean the useless data (not assigned)

df=df[df['Borough']!='Not assigned']

In [10]:
df[df['Neighborhood']=='Not assigned']=df['Borough']
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [11]:
df2=df.groupby('Postal code')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df2=df2.reset_index(drop=False)
df2.rename(columns={'Neighborhood':'Neighborhood_combined'},inplace=True)

In [12]:
df_merge = pd.merge(df, df2, on = 'Postal code')

In [13]:
df_merge.drop(['Neighborhood'], axis=1, inplace=True)

In [14]:
df_merge.drop_duplicates(inplace=True)

In [18]:
df_merge.rename(columns={'Neighborhood_combined':'Neighborhood'}, inplace=True)
df_merge.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


In [20]:
df_merge.shape

(103, 3)

In [21]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [22]:
gloc_df=pd.read_csv('http://cocl.us/Geospatial_data')
gloc_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
gloc_df.rename(columns={'Postal Code':'Postal code'},inplace=True)
gloc_merge = pd.merge(gloc_df, df_merge, on='Postal code')

In [29]:
gloc_data=gloc_merge[['Postal code','Borough','Neighborhood','Latitude','Longitude']]

In [30]:
gloc_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [31]:
#Use of Foursquare app

CLIENT_ID = 'T1YFTS213UT2UDOPUW4FAKRFM5T1XOGG4MO2IGWOLEZZENSN'
CLIENT_SECRET = 'DWVR1W4OICYNXF0FZX3LHYYEZ4BQAN3QSMRYOI3FD2N5BBB0'
VERSION = '20180604'

In [32]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=600
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
# get the infos for Toronto
toronto_venues = getNearbyVenues(names=gloc_data['Neighborhood'],
                                   latitudes=gloc_data['Latitude'],
                                   longitudes=gloc_data['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [35]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
2,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
3,"Rouge, Malvern",43.806686,-79.194353,The Full Custom,43.804940,-79.200482,Furniture / Home Store
4,"Rouge, Malvern",43.806686,-79.194353,Eco Painting,43.807465,-79.201223,Paintball Field


In [36]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,6,6,6,6,6,6
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",6,6,6,6,6,6
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",13,13,13,13,13,13
"Alderwood, Long Branch",12,12,12,12,12,12
"Bathurst Manor, Downsview North, Wilson Heights",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",29,29,29,29,29,29
Berczy Park,89,89,89,89,89,89


In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(2844, 296)

In [38]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,...,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Cosmetics Shop,Sushi Restaurant,Pizza Place,Gym,Asian Restaurant,Theater,Restaurant,Furniture / Home Store,Hotel,Breakfast Spot
1,Agincourt,Lounge,Intersection,Clothing Store,Latin American Restaurant,Skating Rink,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Yoga Studio,Donut Shop,Dessert Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Fast Food Restaurant,BBQ Joint,Park,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Yoga Studio,Deli / Bodega,Donut Shop,Drugstore
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Gym Pool,Fried Chicken Joint,Fast Food Restaurant,Beer Store,Sandwich Place,Auto Garage,Liquor Store,Coffee Shop,Hardware Store,Pharmacy,Pizza Place,Dog Run,Dim Sum Restaurant,Diner
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pool,Skating Rink,Sandwich Place,Convenience Store,Pub,Gas Station,Market,Gym,Pharmacy,College Stadium,Curling Ice,Donut Shop,Doner Restaurant


In [44]:
!pip install folium

     |████████████████████████████████| 92kB 12.8MB/s eta 0:00:01


In [65]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [66]:
gloc_merge = gloc_merge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


ValueError: columns overlap but no suffix specified: Index(['Cluster Labels', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue',
       '11th Most Common Venue', '12th Most Common Venue',
       '13th Most Common Venue', '14th Most Common Venue',
       '15th Most Common Venue'],
      dtype='object')

In [67]:
gloc_merge.head()

,Postal code,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern",0.0,Fast Food Restaurant,Hobby Shop,Furniture / Home Store,Paintball Field,...,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Yoga Studio,Doner Restaurant,Dance Studio
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union",0.0,Moving Target,Bar,Construction & Landscaping,Empanada Restaurant,...,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Deli / Bodega,Doner Restaurant,Ethiopian Restaurant,Dog Run,Dive Bar,Discount Store
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill",0.0,Pizza Place,Fast Food Restaurant,Mexican Restaurant,Electronics Store,...,Park,Intersection,Breakfast Spot,Medical Center,Fried Chicken Joint,Rental Car Location,Bus Line,Thrift / Vintage Store,Dim Sum Restaurant,Diner
3,M1G,43.770992,-79.216917,Scarborough,Woburn,0.0,Coffee Shop,Business Service,Korean Restaurant,Doner Restaurant,...,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Yoga Studio,Donut Shop,Department Store,Dumpling Restaurant,Eastern European Restaurant
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae,0.0,Indian Restaurant,Bakery,Chinese Restaurant,Flower Shop,...,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Gas Station,Bank,Coffee Shop,Hakka Restaurant,Dog Run,Dive Bar,Doner Restaurant


In [68]:
neighborhoods_venues_sorted.head(15)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Cosmetics Shop,Sushi Restaurant,Pizza Place,Gym,Asian Restaurant,Theater,Restaurant,Furniture / Home Store,Hotel,Breakfast Spot
1,0,Agincourt,Lounge,Intersection,Clothing Store,Latin American Restaurant,Skating Rink,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Yoga Studio,Donut Shop,Dessert Shop
2,0,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Fast Food Restaurant,BBQ Joint,Park,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Yoga Studio,Deli / Bodega,Donut Shop,Drugstore
3,0,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Gym Pool,Fried Chicken Joint,Fast Food Restaurant,Beer Store,Sandwich Place,Auto Garage,Liquor Store,Coffee Shop,Hardware Store,Pharmacy,Pizza Place,Dog Run,Dim Sum Restaurant,Diner
4,0,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pool,Skating Rink,Sandwich Place,Convenience Store,Pub,Gas Station,Market,Gym,Pharmacy,College Stadium,Curling Ice,Donut Shop,Doner Restaurant
5,0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Deli / Bodega,Sushi Restaurant,Supermarket,Shopping Mall,Diner,Bridal Shop,Sandwich Place,Middle Eastern Restaurant,Ice Cream Shop,Restaurant,Community Center,Chinese Restaurant,Pizza Place,Pet Store
6,0,Bayview Village,Chinese Restaurant,Café,Japanese Restaurant,Bank,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Yoga Studio,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
7,0,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Fast Food Restaurant,Pet Store,Thai Restaurant,Liquor Store,Juice Bar,Butcher,Restaurant,Café,Indian Restaurant,Sushi Restaurant,Bakery,Bagel Shop
8,0,Berczy Park,Coffee Shop,Seafood Restaurant,Café,Cocktail Bar,Restaurant,Japanese Restaurant,Hotel,Lounge,Cheese Shop,Sporting Goods Shop,Bakery,Beer Bar,Farmers Market,Creperie,Steakhouse
9,0,"Birch Cliff, Cliffside West",General Entertainment,Café,College Stadium,Diner,Skating Rink,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Deli / Bodega,Donut Shop,Drugstore


In [54]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [57]:
import folium

In [62]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(gloc_merge['Latitude'], gloc_merge['Longitude'], gloc_merge['Neighborhood'], gloc_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='grey',
        fill_opacity=0.4).add_to(map_clusters)
       
map_clusters